# APTOS 2019 Blindness Detection

ref : https://www.kaggle.com/c/aptos2019-blindness-detection

## Summary

Model : MobileNetV2

ref :
- https://arxiv.org/pdf/1801.04381.pdf
- https://pytorch.org/hub/pytorch_vision_mobilenet_v2/

TODO
- fastprogress > 自分の手元ではうまく行かず
- log > script化と合わせて
- visualize > 
- script

In [1]:
# SEED
SEED = 1116

# CNN
epochs = 100
verbose = 1
early_stopping_rounds = 10
train_batch_size = 16
valid_batch_size = 16

In [2]:
## system
import sys
import os
import copy
import random
import pathlib

## numeric processing
import numpy as np
import pandas as pd

## image processing
import cv2
from PIL import Image

## machine learning
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score

## pytorch
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

## util
from itertools import chain
from tqdm import tqdm_notebook as tqdm

## visualize
import matplotlib.pyplot as plt
import seaborn as sns

## Initialize

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
VersionName = "mobile0000"
# seed
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## DataLoader

### Dataset

In [4]:
transform_dict = {
    "train": transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    "valid": transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    "test": transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
}

In [5]:
class APTOS2019DatasetTrain(Dataset):
    
    def __init__(self, df, train_path="../input/train_images", transforms=None):
        self.data = df
        self.transforms = transforms
        self.train_path = train_path
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.train_path, self.data.loc[idx, "id_code"]+".png")
        img = cv2.imread(img_path)
        
        label = torch.tensor(self.data.loc[idx, "diagnosis"])
        
        sample = {"image": img, "label": label}
        if self.transforms:
            sample["image"] = self.transforms(sample["image"])
        else:
            sample["image"] = Image.fromarray(np.uint8(sample["image"]))
            sample["image"] = transforms.ToTensor()(sample["image"])
        
        return sample

### Validation

In [6]:
train_df = pd.read_csv("../input/train.csv")

train, valid = train_test_split(train_df, test_size=0.2, stratify=train_df["diagnosis"], random_state=SEED)
train.reset_index(inplace=True)
valid.reset_index(inplace=True)

train_loader = DataLoader(APTOS2019DatasetTrain(train, transforms=transform_dict["train"]),
                          batch_size=train_batch_size,
                          shuffle=False)
valid_loader = DataLoader(APTOS2019DatasetTrain(valid, transforms=transform_dict["valid"]),
                          batch_size=valid_batch_size,
                          shuffle=False)

## Model

In [1]:
import torch
model = torch.hub.load('pytorch/vision', 'mobilenet_v2', pretrained=True)
torch.save(model.state_dict(), "../model/pretrain/mobilenet_v2.pth")
print(model)

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_

Using cache found in /home/icebee/.cache/torch/hub/pytorch_vision_master


In [7]:
model = torchvision.models.mobilenet_v2()
model.load_state_dict(torch.load("../model/pretrain/mobilenet_v2.pth"))
model.classifier[1] = nn.Linear(in_features=1280, out_features=1)
model.to(device)

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_

## Define metrics

In [8]:
# classification module
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.):
        super().__init__()
        self.gamma = gamma

    def forward(self, inputs, targets, **kwargs):
        CE_loss = nn.CrossEntropyLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-CE_loss)
        F_loss = ((1 - pt)**self.gamma) * CE_loss
        return F_loss.mean()

## Training

In [8]:
plist = [
    {'params': model.features.parameters(), 'lr': 1e-4, 'weight': 0.001},
    {'params': model.classifier.parameters(), 'lr': 1e-3}
]
optimizer = optim.Adam(plist, lr=0.001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=10)
criterion = nn.MSELoss()

In [ ]:
best_score = {
    "train": np.Inf,
    "valid": np.Inf
}
best_model_weights = copy.deepcopy(model.state_dict())
learning_list = []

non_improvement_round = 0
for epoch in range(epochs):
    temp_score = {"epoch": epoch, "train": 0.0, "valid": 0.0}
    for phase in ["train", "valid"]:
        if phase == "train":
            data_loader = train_loader
            scheduler.step()
            model.train()
        else:
            data_loader = valid_loader
            model.eval()
            
        running_loss = 0
        for data in tqdm(data_loader):
            inputs = data["image"].to(device, dtype=torch.float)
            labels = data["label"].view(-1, 1).to(device, dtype=torch.float)
            optimizer.zero_grad()
            outputs = model(inputs)
            with torch.set_grad_enabled(phase == "train"):
                loss = criterion(outputs, labels)
                if phase == "train":
                    loss.backward()
                    optimizer.step()
            running_loss += loss.item()
            if torch.cuda.is_available():
                labels = labels.cpu()
                outputs = outputs.cpu()
            y_true = labels.detach().numpy()
            y_pred = outputs.detach()
        temp_score["{}".format(phase)] = running_loss / len(data_loader)
    
    learning_list.append(temp_score)
    # Update scores
    if best_score["train"] > temp_score["train"]:
        best_score["train"] = temp_score["train"]
        best_score["valid"] = temp_score["valid"]
        best_model_weights = copy.deepcopy(model.state_dict())
        non_improvement_round = 0
    else:
        non_improvement_round += 1
    
    # Display
    if epoch % verbose == 0 and verbose != -1:
        print("\t Epoch: {}/{}, Train: {}, Valid: {}".format(
            epoch, epochs-1, temp_score["train"], temp_score["valid"]
        ))
    
    # Early stopping
    if non_improvement_round >= early_stopping_rounds:
        print("\t Early stopping {}/{}".format(epoch, epochs-1))
        break

model.load_state_dict(best_model_weights)
torch.save(model.state_dict(), "../model/{}.pth".format(VersionName))


	 Epoch: 0/99, Train: 0.7420329356404103, Valid: 0.3242992522275966



	 Epoch: 1/99, Train: 0.4257648947000828, Valid: 0.39920504358799563



	 Epoch: 2/99, Train: 0.2814415301317754, Valid: 0.36125404112364934



	 Epoch: 3/99, Train: 0.23628150249588425, Valid: 0.3638870001811048



	 Epoch: 4/99, Train: 0.17544049739270753, Valid: 0.3632070835193862


...


	 Epoch: 7/99, Train: 0.14597230063467895, Valid: 0.38891587033867836



	 Epoch: 8/99, Train: 0.14718246804698973, Valid: 0.4150828997726026



	 Epoch: 9/99, Train: 0.09121328171686796, Valid: 0.294013831116583



	 Epoch: 10/99, Train: 0.06251193780664598, Valid: 0.2911447613783505



	 Epoch: 11/99, Train: 0.053469489303255534, Valid: 0.2856395846313756


In [ ]:
epochs = [t["epoch"] for t in learning_list]
train_loss = [t["train"] for t in learning_list]
valid_loss = [t["valid"] for t in learning_list]

plt.plot(epochs, valid_loss)
plt.plot(epochs, train_loss)
plt.xlim((0, epochs[-1]))
plt.ylim((0, 20))
plt.show()